Implementation of DGCNN Model using PyTorch Geometric and Weights & Biases 🪄🐝





This notebook demonstrates an implementation of the [Dynamic Graph CNN](https://arxiv.org/pdf/1801.07829.pdf) for point cloud segmnetation implemented using [PyTorch Geometric](https://www.pyg.org/) and experiment tracked and visualized using [Weights & Biases](https://wandb.ai/site). The code here is inspired by [this](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/dgcnn_segmentation.py) original implementation.





# Install Required Packages

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.1.0+cu121


In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00


## Import Libraries

In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 5.8 MB/s eta 0:00:00


In [ ]:
import os
import wandb
import random
import numpy as np
from tqdm.auto import tqdm

import torch
import torch.nn.functional as F

from torch_scatter import scatter
from torchmetrics.functional import jaccard_index

import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, DynamicEdgeConv

# Initialize Weights & Biases

We need to call [`wandb.init()`](https://docs.wandb.ai/ref/python/init) once at the beginning of our program to initialize a new job. This creates a new run in W&B and launches a background process to sync data.

In [ ]:
wandb_project = "FDS_Miniproject_Chairs" #@param {"type": "string"}
wandb_run_name = "FDS_Miniproject_Chairs" #@param {"type": "string"}

wandb.init(project=wandb_project, name=wandb_run_name, job_type="train")

config = wandb.config

config.seed = 42
config.device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(config.seed)
torch.manual_seed(config.seed)
device = torch.device(config.device)

config.category = "Chair" #@param ["Bag","Airplane","Cap", "Car", "Chair", "Earphone", "Guitar", "Knife", "Lamp", "Laptop", "Motorbike", "Mug", "Pistol", "Rocket", "Skateboard", "Table"] {type:"raw"}
config.random_jitter_translation = 2e-2
config.random_rotation_interval_x = 15
config.random_rotation_interval_y = 15
config.random_rotation_interval_z = 15
config.validation_split = 0.2  # 80-20 split
config.batch_size = 16
config.num_workers = 6

config.num_nearest_neighbours = 50
config.aggregation_operator = "max"
config.dropout = 0.5
config.initial_lr = 1e-3
config.lr_scheduler_step_size = 5
config.gamma = 0.8

config.epochs = 15

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Load ShapeNet Dataset using PyTorch Geometric

We now load, preprocess and batch the ModelNet dataset for training, validation/testing and visualization.

In [ ]:
transform = T.Compose([
    T.RandomJitter(config.random_jitter_translation),
    T.RandomRotate(config.random_rotation_interval_x, axis=0),
    T.RandomRotate(config.random_rotation_interval_y, axis=1),
    T.RandomRotate(config.random_rotation_interval_z, axis=2)
])
pre_transform = T.NormalizeScale()

In [ ]:
dataset_path = os.path.join('ShapeNet', config.category)

train_val_dataset = ShapeNet(
    dataset_path, config.category, split='trainval',
    transform=transform, pre_transform=pre_transform
)

Extracting ShapeNet/Chair/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!


In [ ]:
train_val_dataset[0]

Data(x=[2736, 3], y=[2736], pos=[2736, 3], category=[1])

In [ ]:
train_val_dataset

ShapeNet(3054, categories=['Chair'])

In [ ]:
train_val_dataset[1]

Data(x=[2500, 3], y=[2500], pos=[2500, 3], category=[1])

In [ ]:
len(train_val_dataset)

3054

In [ ]:
type(train_val_dataset)

torch_geometric.datasets.shapenet.ShapeNet

In [ ]:
from torch_geometric.datasets import ShapeNet

# Load the ShapeNet dataset
dataset = ShapeNet(root='dataset_path', categories=['Airplane'])

# Print dataset information
print(dataset)

# Access individual data samples
data = dataset[0]
print(data)


Extracting dataset_path/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...


ShapeNet(2349, categories=['Airplane'])
Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])


Done!


In [ ]:
data['x']

tensor([[-0.0392,  0.3344,  0.9416],
        [ 0.0011,  0.3488, -0.9372],
        [-0.2507,  0.9366,  0.2447],
        ...,
        [ 0.6270, -0.5863,  0.5130],
        [-0.2090,  0.9760, -0.0607],
        [-0.2459,  0.9653, -0.0878]])

In [ ]:
data.items()

ItemsView({'x': tensor([[-0.0392,  0.3344,  0.9416],
        [ 0.0011,  0.3488, -0.9372],
        [-0.2507,  0.9366,  0.2447],
        ...,
        [ 0.6270, -0.5863,  0.5130],
        [-0.2090,  0.9760, -0.0607],
        [-0.2459,  0.9653, -0.0878]]), 'y': tensor([0, 0, 3,  ..., 3, 1, 1]), 'pos': tensor([[-0.0145, -0.0164,  0.0320],
        [-0.0119, -0.0657,  0.0145],
        [-0.1424, -0.0370, -0.0519],
        ...,
        [ 0.0342, -0.0931, -0.0523],
        [-0.0108, -0.0600,  0.0522],
        [-0.0165, -0.0593,  0.0560]]), 'category': tensor([0])})

In [ ]:
type(data)

torch_geometric.data.data.Data

Now, we need to offset the segmentation labels

In [ ]:
segmentation_class_frequency = {}
for idx in tqdm(range(len(train_val_dataset))):
    pc_viz = train_val_dataset[idx].pos.numpy().tolist()
    segmentation_label = train_val_dataset[idx].y.numpy().tolist()
    for label in set(segmentation_label):
        segmentation_class_frequency[label] = segmentation_label.count(label)
class_offset = min(list(segmentation_class_frequency.keys()))
print("Class Offset:", class_offset)

for idx in range(len(train_val_dataset)):
    train_val_dataset[idx].y -= class_offset

  0%|          | 0/3054 [00:00<?, ?it/s]

Class Offset: 12


In [ ]:
num_train_examples = int((1 - config.validation_split) * len(train_val_dataset))
train_dataset = train_val_dataset[:num_train_examples]
val_dataset = train_val_dataset[num_train_examples:]

In [ ]:
train_dataset[0]

Data(x=[2736, 3], y=[2736], pos=[2736, 3], category=[1])

In [ ]:
len(train_dataset)

2443

In [ ]:
train_loader = DataLoader(
    train_dataset, batch_size=config.batch_size,
    shuffle=True, num_workers=config.num_workers
)
val_loader = DataLoader(
    val_dataset, batch_size=config.batch_size,
    shuffle=False, num_workers=config.num_workers
)
visualization_loader = DataLoader(
    val_dataset[:10], batch_size=1,
    shuffle=False, num_workers=config.num_workers
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# Implementing the DGCNN Model using PyTorch Geometric

In [ ]:
class DGCNN(torch.nn.Module):
    def __init__(self, out_channels, k=30, aggr='max'):
        super().__init__()

        self.conv1 = DynamicEdgeConv(MLP([2 * 6, 64, 64]), k, aggr)
        self.conv2 = DynamicEdgeConv(MLP([2 * 64, 64, 64]), k, aggr)
        self.conv3 = DynamicEdgeConv(MLP([2 * 64, 64, 64]), k, aggr)

        self.mlp = MLP(
            [3 * 64, 1024, 256, 128, out_channels],
            dropout=0.5, norm=None
        )

    def forward(self, data):
        x, pos, batch = data.x, data.pos, data.batch
        x0 = torch.cat([x, pos], dim=-1)

        x1 = self.conv1(x0, batch)
        x2 = self.conv2(x1, batch)
        x3 = self.conv3(x2, batch)

        out = self.mlp(torch.cat([x1, x2, x3], dim=1))
        return F.log_softmax(out, dim=1)

In [ ]:
train_dataset.num_classes

50

In [ ]:
config.num_classes = train_dataset.num_classes

model = DGCNN(
    out_channels=train_dataset.num_classes,
    k=config.num_nearest_neighbours,
    aggr=config.aggregation_operator
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=config.lr_scheduler_step_size, gamma=config.gamma
)

# Training DGCNN and Logging Metrics on Weights & Biases

In [ ]:
def train_step(epoch):
    model.train()

    ious, categories = [], []
    total_loss = correct_nodes = total_nodes = 0
    y_map = torch.empty(
        train_loader.dataset.num_classes, device=device
    ).long()
    num_train_examples = len(train_loader)

    progress_bar = tqdm(
        train_loader, desc=f"Training Epoch {epoch}/{config.epochs}"
    )

    for data in progress_bar:
        data = data.to(device)

        optimizer.zero_grad()
        outs = model(data)
        loss = F.nll_loss(outs, data.y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        correct_nodes += outs.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        sizes = (data.ptr[1:] - data.ptr[:-1]).tolist()
        for out, y, category in zip(outs.split(sizes), data.y.split(sizes),
                                    data.category.tolist()):
            category = list(ShapeNet.seg_classes.keys())[category]
            part = ShapeNet.seg_classes[category]
            part = torch.tensor(part, device=device)

            y_map[part] = torch.arange(part.size(0), device=device)

            iou = jaccard_index(
                out[:, part].argmax(dim=-1), y_map[y],
                task="multiclass", num_classes=part.size(0)
            )
            ious.append(iou)

        categories.append(data.category)

    iou = torch.tensor(ious, device=device)
    category = torch.cat(categories, dim=0)
    mean_iou = float(scatter(iou, category, reduce='mean').mean())

    return {
        "Train/Loss": total_loss / num_train_examples,
        "Train/Accuracy": correct_nodes / total_nodes,
        "Train/IoU": mean_iou
    }

In [ ]:
@torch.no_grad()
def val_step(epoch):
    model.eval()

    ious, categories = [], []
    total_loss = correct_nodes = total_nodes = 0
    y_map = torch.empty(
        val_loader.dataset.num_classes, device=device
    ).long()
    num_val_examples = len(val_loader)

    progress_bar = tqdm(
        val_loader, desc=f"Validating Epoch {epoch}/{config.epochs}"
    )

    for data in progress_bar:
        data = data.to(device)
        outs = model(data)

        loss = F.nll_loss(outs, data.y)
        total_loss += loss.item()

        correct_nodes += outs.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        sizes = (data.ptr[1:] - data.ptr[:-1]).tolist()
        for out, y, category in zip(outs.split(sizes), data.y.split(sizes),
                                    data.category.tolist()):
            category = list(ShapeNet.seg_classes.keys())[category]
            part = ShapeNet.seg_classes[category]
            part = torch.tensor(part, device=device)

            y_map[part] = torch.arange(part.size(0), device=device)

            iou = jaccard_index(
                out[:, part].argmax(dim=-1), y_map[y],
                task="multiclass", num_classes=part.size(0)
            )
            ious.append(iou)

        categories.append(data.category)

    iou = torch.tensor(ious, device=device)
    category = torch.cat(categories, dim=0)
    mean_iou = float(scatter(iou, category, reduce='mean').mean())

    return {
        "Validation/Loss": total_loss / num_val_examples,
        "Validation/Accuracy": correct_nodes / total_nodes,
        "Validation/IoU": mean_iou
    }

In [ ]:
@torch.no_grad()
def visualization_step(epoch, table):
    model.eval()
    for data in tqdm(visualization_loader):
        data = data.to(device)
        outs = model(data)

        predicted_labels = outs.argmax(dim=1)
        accuracy = predicted_labels.eq(data.y).sum().item() / data.num_nodes

        sizes = (data.ptr[1:] - data.ptr[:-1]).tolist()
        ious, categories = [], []
        y_map = torch.empty(
            visualization_loader.dataset.num_classes, device=device
        ).long()
        for out, y, category in zip(
            outs.split(sizes), data.y.split(sizes), data.category.tolist()
        ):
            category = list(ShapeNet.seg_classes.keys())[category]
            part = ShapeNet.seg_classes[category]
            part = torch.tensor(part, device=device)
            y_map[part] = torch.arange(part.size(0), device=device)
            iou = jaccard_index(
                out[:, part].argmax(dim=-1), y_map[y],
                task="multiclass", num_classes=part.size(0)
            )
            ious.append(iou)
        categories.append(data.category)
        iou = torch.tensor(ious, device=device)
        category = torch.cat(categories, dim=0)
        mean_iou = float(scatter(iou, category, reduce='mean').mean())

        gt_pc_viz = data.pos.cpu().numpy().tolist()
        segmentation_label = data.y.cpu().numpy().tolist()
        frequency_dict = {key: 0 for key in segmentation_class_frequency.keys()}
        for label in set(segmentation_label):
            frequency_dict[label] = segmentation_label.count(label)
        for j in range(len(gt_pc_viz)):
            # gt_pc_viz[j] += [segmentation_label[j] + 1 - class_offset]
            gt_pc_viz[j] += [segmentation_label[j] + 1]

        predicted_pc_viz = data.pos.cpu().numpy().tolist()
        segmentation_label = data.y.cpu().numpy().tolist()
        frequency_dict = {key: 0 for key in segmentation_class_frequency.keys()}
        for label in set(segmentation_label):
            frequency_dict[label] = segmentation_label.count(label)
        for j in range(len(predicted_pc_viz)):
            # predicted_pc_viz[j] += [segmentation_label[j] + 1 - class_offset]
            predicted_pc_viz[j] += [segmentation_label[j] + 1]

        table.add_data(
            epoch, wandb.Object3D(np.array(gt_pc_viz)),
            wandb.Object3D(np.array(predicted_pc_viz)),
            accuracy, mean_iou
        )

    return table

In [ ]:
def save_checkpoint(epoch):
    """Save model checkpoints as Weights & Biases artifacts"""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, "checkpoint.pt")

    artifact_name = wandb.util.make_artifact_name_safe(
        f"{wandb.run.name}-{wandb.run.id}-checkpoint"
    )

    checkpoint_artifact = wandb.Artifact(artifact_name, type="checkpoint")
    checkpoint_artifact.add_file("checkpoint.pt")
    wandb.log_artifact(
        checkpoint_artifact, aliases=["latest", f"epoch-{epoch}"]
    )

In [ ]:
table = wandb.Table(columns=["Epoch", "Ground-Truth", "Prediction", "Accuracy", "IoU"])

best_val_accuracy = 0.90
#stop_training_threshold = 0.97  # Set your desired accuracy threshold to stop training

for epoch in range(1, config.epochs + 1):
    train_metrics = train_step(epoch)
    val_metrics = val_step(epoch)
    metrics = {**train_metrics, **val_metrics}
    metrics["learning_rate"] = scheduler.get_last_lr()[-1]
    wandb.log(metrics)

    # Check if the validation accuracy increased
    if val_metrics["Validation/Accuracy"] > best_val_accuracy:
        best_val_accuracy = val_metrics["Validation/Accuracy"]

        # Save the model checkpoint
        save_checkpoint(epoch)

    # Check if the validation accuracy is below the stop threshold
    if (val_metrics["Validation/Accuracy"] < best_val_accuracy*0.98) and (val_metrics["Validation/Accuracy"]>0.95)  :
        print(f"Stopping training as validation accuracy dropped below {stop_training_threshold}")
        break

    # Visualization step
    table = visualization_step(epoch, table)

    # Learning rate scheduler step
    scheduler.step()


wandb.log({"Evaluation": table})


Training Epoch 1/15:   0%|          | 0/153 [00:00<?, ?it/s]

Validating Epoch 1/15:   0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Training Epoch 2/15:   0%|          | 0/153 [00:00<?, ?it/s]

Validating Epoch 2/15:   0%|          | 0/39 [00:00<?, ?it/s]

...

Training Epoch 12/15:   0%|          | 0/153 [00:00<?, ?it/s]

Validating Epoch 12/15:   0%|          | 0/39 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c2b71d772e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c2b71d772e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/

  0%|          | 0/10 [00:00<?, ?it/s]

Training Epoch 13/15:   0%|          | 0/153 [00:00<?, ?it/s]

In [ ]:
wandb.log(metrics)

In [ ]:
wandb.finish()

#### The Performance Metrics of DGCNN on ShapeNet Dataset

https://api.wandb.ai/links/team-dubakur/638pjq2p

In [ ]:
print("completed")